In [21]:
import pandas as pd
import requests
from keys import *

In [26]:
keys = getKeys()

In [27]:
url = "https://us-real-estate.p.rapidapi.com/v2/property-detail"

querystring = {"property_id":"3199790641"}

headers = {
    "X-RapidAPI-Key": keys['USRealEstate'],
    "X-RapidAPI-Host": "us-real-estate.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"status":200,"data":{"property_detail":{"photos":[{"href":"https:\/\/maps.googleapis.com\/maps\/api\/streetview?channel=rdc-streetview&client=gme-movesalesinc&size=640x480&location=19360+Murray+Hill+St%2CDetroit%2CMI%2C48235&signature=KDmlbkncYjq4jvYNDHe7PdHXRf4"}],"forwarded_property_ids":{},"mpr_id":3199790641,"address":{"street_post_direction":null,"state":"Michigan","address_validation_code":"121","line":"19360 Murray Hill St","boundary":{"coordinates":[-83.208158,42.433453],"type":"Point"},"street_number":"19360","street_suffix":"St","time_zone":"America\/New_York","state_code":"MI","county":"Wayne","street_direction":null,"postal_code":"48235","county_needed_for_uniq":false,"street_name":"Murray Hill","country":"USA","city":"Detroit","location":{"lon":-83.208158,"lat":42.433453},"unit_value":null},"buyer_agent_for_far":{},"property_overview":"19360 Murray Hill St, Detroit, MI 48235 is a single family home built in 1948. This property was last sold for $115,000 in 2021 and curren